# Test KFP Integration

- create an experiment
- create a run
- check run passes

This Notebook is based on the condition Kubeflow pipelines [example](https://github.com/kubeflow/pipelines/blob/master/samples/core/condition/condition_v2.py).

In [ ]:
# Please check the requirements.in file for more details
!pip install -r requirements.txt

In [ ]:
import kfp
import os

from kfp import dsl
from tenacity import retry, stop_after_attempt, wait_exponential

In [ ]:
client = kfp.Client()

In [ ]:
EXPERIMENT_NAME = "Flip a coin and output tails/heads pipeline"

In [ ]:
HTTP_PROXY = HTTPS_PROXY = NO_PROXY = None

if os.environ.get("HTTP_PROXY") and os.environ.get("HTTPS_PROXY") and os.environ.get("NO_PROXY"):
    HTTP_PROXY = os.environ["HTTP_PROXY"]
    HTTPS_PROXY = os.environ["HTTPS_PROXY"]
    NO_PROXY = os.environ["NO_PROXY"]

In [ ]:
def add_proxy(obj, http_proxy=HTTP_PROXY, https_proxy=HTTPS_PROXY, no_proxy=NO_PROXY):
    """Adds the proxy env vars to the PipelineTask object."""
    return (
        obj.set_env_variable(name="http_proxy", value=http_proxy)
        .set_env_variable(name="https_proxy", value=https_proxy)
        .set_env_variable(name="HTTP_PROXY", value=http_proxy)
        .set_env_variable(name="HTTPS_PROXY", value=https_proxy)
        .set_env_variable(name="no_proxy", value=no_proxy)
        .set_env_variable(name="NO_PROXY", value=no_proxy)
    )

In [ ]:
def proxy_envs_set():
    if HTTP_PROXY and HTTPS_PROXY and NO_PROXY:
        return True
    return False

In [ ]:
@dsl.component()
def flip_coin(force_flip_result: str = "") -> str:
    """Flip a coin and output heads or tails randomly."""
    if force_flip_result:
        return force_flip_result
    import random

    result = "heads" if random.randint(0, 1) == 0 else "tails"
    return result

In [ ]:
@dsl.component()
def print_msg(msg: str):
    """Print a message."""
    print(msg)

In [ ]:
@dsl.pipeline(name="condition-v2")
def condition_pipeline(text: str = "condition test", force_flip_result: str = ""):
    flip1 = flip_coin(force_flip_result=force_flip_result)
    print_msg(msg=flip1.output)

    with dsl.Condition(flip1.output == "heads"):
        flip2 = flip_coin()
        print_msg(msg=flip2.output)
        print_msg(msg=text)

In [ ]:
@dsl.pipeline(name="condition-v2")
def condition_pipeline_proxy(text: str = "condition test", force_flip_result: str = ""):
    flip1 = add_proxy(flip_coin(force_flip_result=force_flip_result))
    add_proxy(print_msg(msg=flip1.output))

    with dsl.Condition(flip1.output == "heads"):
        flip2 = add_proxy(flip_coin())
        add_proxy(print_msg(msg=flip2.output))
        add_proxy(print_msg(msg=text))

In [ ]:
# Setting enable_caching to False to overcome https://github.com/canonical/bundle-kubeflow/issues/1067
if proxy_envs_set():
    run = client.create_run_from_pipeline_func(
        condition_pipeline_proxy,
        experiment_name=EXPERIMENT_NAME,
        enable_caching=False,
    )
else:
    run = client.create_run_from_pipeline_func(
        condition_pipeline,
        experiment_name=EXPERIMENT_NAME,
        enable_caching=False,
    )

In [ ]:
client.list_experiments().experiments

In [ ]:
client.get_run(run.run_id).state

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_run_succeeded(client, run_id):
    """Wait for the run to complete successfully."""
    status = client.get_run(run_id).state
    assert status == "SUCCEEDED", f"KFP run in {status} state."

In [ ]:
# fetch KFP experiment to ensure it exists
client.get_experiment(experiment_name=EXPERIMENT_NAME)

assert_run_succeeded(client, run.run_id)